In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
sys.path.append('../../../../nadavp/opt_lev_analysis/scripts/calibration/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [6]:
step_cal_dir = r'/data/old_trap/20210429/bead1/discharge/fine_after_recharge/'
max_file = 500

In [7]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
len(step_cal_files)

135

In [8]:
# Do the step calibration
step_file_objs = []
for filname in step_cal_files[1:max_file]:
    df = bu.DataFile();
    df.load(filname);
    df.load_other_data()
    step_file_objs.append(df);

In [10]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [11]:
func1_scale(0)

11164.571270053859

34.079267745559036

In [18]:
amps3 = []
for i in range(134):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 135
1 / 135
2 / 135
3 / 135
4 / 135
5 / 135
6 / 135
7 / 135
8 / 135
9 / 135
10 / 135
11 / 135
12 / 135
13 / 135
14 / 135
15 / 135
16 / 135
17 / 135
18 / 135
19 / 135
20 / 135
21 / 135
22 / 135
23 / 135
24 / 135
25 / 135
26 / 135
27 / 135
28 / 135
29 / 135
30 / 135
31 / 135
32 / 135
33 / 135
34 / 135
35 / 135
36 / 135
37 / 135
38 / 135
39 / 135
40 / 135
41 / 135
42 / 135
43 / 135
44 / 135
45 / 135
46 / 135
47 / 135
48 / 135
49 / 135
50 / 135
51 / 135
52 / 135
53 / 135
54 / 135
55 / 135
56 / 135
57 / 135
58 / 135
59 / 135
60 / 135
61 / 135
62 / 135
63 / 135
64 / 135
65 / 135
66 / 135
67 / 135
68 / 135
69 / 135
70 / 135
71 / 135
72 / 135
73 / 135
74 / 135
75 / 135
76 / 135
77 / 135
78 / 135
79 / 135
80 / 135
81 / 135
82 / 135
83 / 135
84 / 135
85 / 135
86 / 135
87 / 135
88 / 135
89 / 135
90 / 135
91 / 135
92 / 135
93 / 135
94 / 135
95 / 135
96 / 135
97 / 135
98 / 135
99 / 135
100 / 135
101 / 135
102 / 135
103 / 135
104 / 135
105 / 135
106 / 135
107 / 135
108 / 135
109 / 135
110 / 135


In [19]:
_,ax = plt.subplots(figsize=(9.5,4))
_ = ax.scatter(range(len(amps3)), amps3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
scale = 1200

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-0.3,11), xlim=(0,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
scale = np.abs(np.mean(amps3[20:55]))/8
print("Scale is ", scale)
bb = step_file_objs[25]
voltage = np.std(bb.other_data[-1])*np.sqrt(2)*50
print("voltage: ", voltage)

Scale is  1207.53645905
voltage:  34.0792677456


### No phase response

In [68]:
df = step_file_objs[50]
[np.std(dd)*np.sqrt(2)*50 for dd in df.other_data]

[0.36632550176441686,
 0.018925587383700471,
 0.034380070438748404,
 0.036843452933939938,
 34.078299512491341]

In [86]:
fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_50V_41_Hz_x/turbombar_powfb_xyzcool_elec3_3000mV41Hz0mVdc_10.h5'
# fname = r'/data/old_trap/20210429/bead1/discharge/fine_after_recharge/turbombar_powfb_xyzcool_elec3_3000mV41Hz0mVdc_10.h5'
del df
df = bu.DataFile();
df.load(fname);
df.load_other_data()
[np.std(dd)*np.sqrt(2)*50 for dd in df.other_data], [np.mean(dd)*50 for dd in df.other_data]

([0.36430330435929487,
  0.019154090256631173,
  0.035212356683502359,
  0.038093320049319389,
  22.71881582353485],
 [-2.0517988860914009,
  -0.17697047379262992,
  -0.24832806897077891,
  -0.25704737926298915,
  -0.23516716258487821])

In [65]:
_, ax = plt.subplots()
ax.scatter(range(5000), df.other_data[-1][:5000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [232]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp(i):
    
#     print(i,'  ******************')
#     fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_10V_bias_x_1/turbombar_powfb_xyzcool_'+str(i)+'.h5'
    fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20210429/bead1/neutrality/w_height_fb_50V_41_Hz_x/turbombar_powfb_xyzcool_elec3_3000mV41Hz0mVdc_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[-1])[::decimate]
#     print(np.std(bb.other_data[-1]))
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[-1]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[-1]))
#     print(m1_tmp.values, (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi))
    phi_tmp = m1_tmp.values[2]
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-10000, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [234]:
i = 100
get_amp(i)
fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
voltage1 = np.std(bb.other_data[-1])*np.sqrt(2)*50
print("voltage: ", voltage1, np.mean(bb.other_data[-1])*50)

voltage:  45.451195226 0.0236881055924


In [235]:
amps_100V_bias_sweep_spin = []
for i in range(1000):
    print(i,'/500')
    try:
        amps_100V_bias_sweep_spin.append(get_amp(i))
    except:
        print("******")
amps_100V_bias_sweep_spin = np.array(amps_100V_bias_sweep_spin)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500
20 /500
21 /500
22 /500
23 /500
24 /500
25 /500
26 /500
27 /500
28 /500
29 /500
30 /500
31 /500
32 /500
33 /500
34 /500
35 /500
36 /500
37 /500
38 /500
39 /500
40 /500
41 /500
42 /500
43 /500
44 /500
45 /500
46 /500
47 /500
48 /500
49 /500
50 /500
51 /500
52 /500
53 /500
54 /500
55 /500
56 /500
57 /500
58 /500
59 /500
60 /500
61 /500
62 /500
63 /500
64 /500
65 /500
66 /500
67 /500
68 /500
69 /500
70 /500
71 /500
72 /500
73 /500
74 /500
75 /500
76 /500
77 /500
78 /500
79 /500
80 /500
81 /500
82 /500
83 /500
84 /500
85 /500
86 /500
87 /500
88 /500
89 /500
90 /500
91 /500
92 /500
93 /500
94 /500
95 /500
96 /500
97 /500
98 /500
99 /500
100 /500
101 /500
102 /500
103 /500
104 /500
105 /500
106 /500
107 /500
108 /500
109 /500
110 /500
111 /500
112 /500
113 /500
114 /500
115 /500
116 /500
117 /500
118 /500
119 /500
120 /500
121 /500
122 /500
123

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_500.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_507.h5
******
508 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_508.h5
******
509 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_509.h5
******
510 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_510.h5
******
511 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_511.h5
******
512 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_561.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_610.h5
******
611 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_611.h5
******
612 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_612.h5
******
613 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_613.h5
******
614 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_614.h5
******
615 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz

...

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_770.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_812.h5
******
813 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_813.h5
******
814 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_814.h5
******
815 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_815.h5
******
816 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_816.h5
******
817 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_872.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_914.h5
******
915 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_915.h5
******
916 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_916.h5
******
917 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_917.h5
******
918 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_918.h5
******
919 /500
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_975.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

In [254]:
data = np.array(amps_100V_bias_sweep_spin)
data2 = np.array(amps_100V_bias_sweep[::2])
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data/scale*voltage/voltage1, label='100V, w/o feedback, sweep, spin 100kHz')
ax.scatter(range(len(data2)), data2/scale*voltage/voltage1, label='100V, w/o feedback, 0.8V bias')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print(np.mean(data/scale*voltage/voltage1), np.std(data/scale*voltage/voltage1)/np.sqrt(len(data)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.00435401644749 0.00154443081035


In [237]:
bias3 = []
for i in range(1000):
    try:
        fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'
        bb = bu.DataFile();
        bb.load(fname);
        bb.load_other_data()
        bias3.append(np.mean(bb.other_data[-1])*50)
    except:
        print('*********')

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_500.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_500.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_501.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_502.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_503.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_504.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_612.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_613.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_614.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_615.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_616.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_617.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_720.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_723.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_724.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_725.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_726.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_727.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbo

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_832.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_834.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_835.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_836.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_837.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_838.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb

*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_949.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_950.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_951.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_952.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_953.h5
*********
Can't find/open HDF5 file : /data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbo

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_945.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45,

In [243]:
bias3

[0.20558266575112527,
 0.2079703974975203,
 0.20077790493629347,
 0.19985259784847781,
 0.19477073319600208,
 0.19695155260547789,
 0.20017517357137404,
 0.18960189211871512,
 0.19375844968337516,
 0.19119920653086125,
 0.18951003280689699,
 0.18511940184634157,
 0.18619607843137251,
 0.18673960479133278,
 0.17982299534599822,
 0.17854215304798954,
 0.1769970244907301,
 0.17560784313725483,
 0.16869764248111691,
 0.17249744411383222,
 0.16935866330968186,
 0.17364248111696035,
 0.16646280613412676,
 0.16177889677271681,
 0.16316502632181268,
 0.16930952925917445,
 0.16068451972228573,
 0.15864255741206984,
 0.1537770656900892,
 0.16003326466773471,
 0.15055191882200342,
 0.14823010605020212,
 0.15544060425726702,
 0.14665384908827342,
 0.14777355611505294,
 0.14393224994277862,
 0.1392925917448691,
 0.13545494773785,
 0.13416433966582736,
 0.14111940184634159,
 0.13612970168612185,
 0.13238941023880371,
 0.13008590829327829,
 0.13262470435645068,
 0.12771190966659035,
 0.12190920881971

In [252]:
_,ax = plt.subplots()
ax.scatter(np.arange(1,-1,-2/1000), bias)
ax.set(xlabel='set voltage [V]', ylabel='measured voltage [V]')
# ax.scatter(np.arange(len(bias2)), bias2)
# ax.scatter(np.arange(len(bias3)), bias3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'set voltage [V]'), Text(0, 0.5, 'measured voltage [V]')]

In [231]:
data = np.array(amps_100V_bias_sweep)
data2 = np.array(amps_100V_bias_sweep_1)
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(bias, data/scale*voltage/voltage1, label='100V, bias sweep, 41Hz.')
ax.scatter(bias2, data2/scale*voltage/voltage1, label='100V, bias sweep, 41Hz.')
# ax.plot(np.arange(1,1000), np.arange(0,1000)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
datasets = [amps_100V_bias_1V[100:], amps_100V_bias_0_8V[100:], amps_100V_bias_0_6V[100:]
           , amps_100V_bias_neg0_6V[100:], amps_100V_bias_neg1V[100:], amps_100V_bias_neg0_8V[100:]
            , amps_100V_bias_neg0_8V_1[100:], amps_100V_bias_0V_1[100:]]
titles = ['1V', '0.8V', '0.6V', '-0.6V', '-1V', '-0.8V', '-0.8V-2' , '0V-2']
voltages = np.array([1, 0.8, 0.6, -0.6, -1, -0.8, -0.8, 0])
voltages = np.array([0.2, 0.12, 0.03, -0.5, -0.7, -0.6, 0.12, -0.24])

amps = []
stds = []

_,ax = plt.subplots(1,2, figsize=(9.4,4))
for dd,t in zip(datasets, titles):
    data =  np.array(dd)/scale*voltage/voltage1
    _ = ax[0].hist(data, bins=50, alpha=0.6, range=(-0.4,0.4), label=t)
    ax[0].set(xlabel=r'$\epsilon$')
    ax[0].legend()
    print(np.mean(data), np.std(data)/np.sqrt(len(data)))
    amps.append(np.mean(dd)/scale*voltage/voltage1)
    stds.append(np.std(dd)/scale*voltage/voltage1)
    
ax[1].errorbar(voltages, amps, yerr=np.array(stds), fmt='o', label='mean')
ax[1].plot(np.arange(-1.1,1.1,0.1), np.arange(-1.1,1.1,0.1)*0, '--r')
# ax[1].scatter(voltages, stds, label='std')
ax[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.192990354447 0.000798032642027
0.187362830019 0.000858494817086
0.170715059575 0.00050453696835
-0.120042963834 0.00512840763216
0.00134299228891 0.00691367464861
0.00905825101226 0.00407565136757
0.174410160478 0.000683678825406
0.119840758764 0.000628550892226


In [208]:
datasets = [amps_100V_bias_0V[100:], amps_100V_bias_neg25V[100:], amps_100V_bias_neg21V[100:], amps_100V_bias_neg8V[100:]
            ,amps_100V_bias_8V[100:], amps_100V_bias_10V[100:], amps_100V_bias_6V[100:]
           , amps_100V_bias_25V[100:]]
titles = ['0V', '-25V', '-21V', '-8V', '8V', '10V', '6V', '25V']
voltages = np.array([0, -25, -21, -8, 8, 10, 6, 25])

amps = []
stds = []

_,ax = plt.subplots(1,2, figsize=(9.4,4))
for dd,t in zip(datasets, titles):
    data =  np.array(dd)/scale*voltage/voltage1
    _ = ax[0].hist(data, bins=50, alpha=0.6, range=(-0.6,0.6), label=t)
    ax[0].set(xlabel=r'$\epsilon$')
    ax[0].legend()
    print(np.mean(data), np.std(data)/np.sqrt(len(data)))
    amps.append(np.mean(dd)/scale*voltage/voltage1)
    stds.append(np.std(dd)/scale*voltage/voltage1)
    
ax[1].errorbar(voltages, amps, yerr=np.array(stds), fmt='o', label='mean')
ax[1].plot(np.arange(-27,27), np.arange(-27,27)*0, '--r')
# ax[1].scatter(voltages, stds, label='std')
ax[1].set(ylabel=r'$\epsilon$')
ax[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.175899682267 0.00368374704652
-0.443582698731 0.00225520624605
-0.455979752249 0.00146476001984
-0.340895894953 0.00150069714009
0.296356721439 0.00320321196385
0.279433539503 0.00191197426337
0.2842820764 0.00149150602059
0.509351569395 0.000890095114995


In [259]:
i=0
fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
freq = np.fft.rfftfreq(len(bb.other_data[0]), d=1./5000)
spin_asd = np.abs(np.fft.rfft(bb.other_data[0]))
_,ax = plt.subplots()
ax.semilogy(freq, spin_asd)

j=10
fname = r'/data/old_trap/20210429/bead1/neutrality/wo_height_fb_100V_41_Hz_bias_sweep_x_100kHz_yz_spinning_redo/turbombar_powfb_xyzcool_'+str(j)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
freq2 = np.fft.rfftfreq(len(bb.other_data[0]), d=1./5000)
spin_asd2 = np.abs(np.fft.rfft(bb.other_data[0]))
ax.semilogy(freq2, spin_asd2, alpha=0.4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
data = amps_50V_w_fb/scale
means = [np.mean(data[j*10:(j+1)*10]) for j in range(100)]
stds = [np.std(data[j*10:(j+1)*10]) for j in range(100)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,10), means, yerr=stds/np.sqrt(10),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [83]:
data = amps_50V_wo_fb/scale
means = [np.mean(data[j*10:(j+1)*10]) for j in range(100)]
stds = [np.std(data[j*10:(j+1)*10]) for j in range(100)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,10), means, yerr=stds/np.sqrt(10),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [70]:
data = amps_50V_w_fb/scale
means = [np.mean(data[j*100:(j+1)*100]) for j in range(10)]
stds = [np.std(data[j*100:(j+1)*100]) for j in range(10)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,100), means, yerr=stds/np.sqrt(100),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]